# Tratamiento de nulos en el campo Edad
Siguiendo el trabajo de mi compañero Luca, existen muchos valores nulos en el campo de edad que habrá de tratar.

## Hipótesis
La predicción de la edad solo será la del conductor, que es quién determina lógicamente tiene más correlación con el número de accidentes.

Para predecir el campo de la edad se tendrán en cuenta las siguientes variables del csv Accidentes_20XX
1. La edad de los demás pasajeros: La edad de los demás pasajeros guarda relación con la edad del conductor. Un conductor jóven es más improbable que lleve a pasajeros menores de edad que un conductor más mayor.
2. La fecha y hora del accidente: Es posible que la fecha y la hora guarde alguna relación con la edad del conductor.
3. El distrito: Es posible que algunos distritos tengan conductores más jóvenes y otros conductores más mayores.
4. Género: Es posible que existan distinta distribución por edad en relación con el género.
5. Tipo de accidente: Es posible que los jóvenes tengan más accidentes de un tipo que los mayores.
6. Drogas y alcohol: Es posible que los jóvenes sean más propensos a conducir bajo influencia de drogas que las personas más mayores.
7. Tipo de vehículo: Es posible que los vehículos industriales sean conducidos por personas más mayores

A continuación se seleccionan los campos a tratar en el dataframe y se proporciona una vista previa a fin de refrescar al lector sobre la estructura de los datos

In [70]:
import pandas as pd
from matplotlib.testing.jpl_units import day

accidentes_2022 = pd.read_csv('./Datasets/2022_Accidentalidad.csv', sep = ';')
accidentes_2022.head()

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,...,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,Unnamed: 19,Unnamed: 20
0,2022S000001,01/01/2022,1:30:00,"AVDA. ALBUFERA, 19",19,13,PUENTE DE VALLECAS,Alcance,Despejado,Turismo,...,De 30 a 34 años,Mujer,NaN,NaN,"443359,226","4472082,272",N,NaN,NaN,NaN
1,2022S000001,01/01/2022,1:30:00,"AVDA. ALBUFERA, 19",19,13,PUENTE DE VALLECAS,Alcance,Despejado,Turismo,...,De 45 a 49 años,Hombre,NaN,NaN,"443359,226","4472082,272",N,NaN,NaN,NaN
2,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3,RETIRO,Colisión fronto-lateral,NaN,Motocicleta hasta 125cc,...,De 30 a 34 años,Hombre,NaN,NaN,"441155,351","4474129,588",S,NaN,NaN,NaN
3,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3,RETIRO,Colisión fronto-lateral,NaN,Motocicleta hasta 125cc,...,De 35 a 39 años,Mujer,NaN,NaN,"441155,351","4474129,588",N,NaN,NaN,NaN
4,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3,RETIRO,Colisión fronto-lateral,NaN,Turismo,...,De 40 a 44 años,Hombre,NaN,NaN,"441155,351","4474129,588",N,NaN,NaN,NaN


### Extracción de campos
A continuación, se extraen los campos num_expediente, fecha y hora, distrito, género, tipo de accidente, drogas y alcohol

In [60]:
df_2022 = accidentes_2022.drop(columns=['coordenada_x_utm', 'coordenada_y_utm', 'localizacion', 'numero', 'cod_lesividad', 'Unnamed: 19', 'Unnamed: 20', 'estado_meteorológico', 'lesividad'])
df_2022.head()

,num_expediente,fecha,hora,cod_distrito,distrito,tipo_accidente,tipo_vehiculo,tipo_persona,rango_edad,sexo,positiva_alcohol,positiva_droga
0,2022S000001,01/01/2022,1:30:00,13,PUENTE DE VALLECAS,Alcance,Turismo,Conductor,De 30 a 34 años,Mujer,N,NaN
1,2022S000001,01/01/2022,1:30:00,13,PUENTE DE VALLECAS,Alcance,Turismo,Conductor,De 45 a 49 años,Hombre,N,NaN
2,2022S000002,01/01/2022,0:30:00,3,RETIRO,Colisión fronto-lateral,Motocicleta hasta 125cc,Conductor,De 30 a 34 años,Hombre,S,NaN
3,2022S000002,01/01/2022,0:30:00,3,RETIRO,Colisión fronto-lateral,Motocicleta hasta 125cc,Pasajero,De 35 a 39 años,Mujer,N,NaN
4,2022S000002,01/01/2022,0:30:00,3,RETIRO,Colisión fronto-lateral,Turismo,Conductor,De 40 a 44 años,Hombre,N,NaN


### Limpieza
Existe un dato redundante, que es el distrito. Se sustituirá por su correspondiente código. Pero antes, vamos a verificar que no existe ningún nulo en la columna cod_distrito

In [61]:
df_2022['cod_distrito'].count() == df_2022['num_expediente'].count()

True

Por tanto, existen tantos no nulos como números de expediente en cod_distrito. Podemos eliminar la columna distrito.

In [62]:
df_2022.pop('distrito')
df_2022.head()

,num_expediente,fecha,hora,cod_distrito,tipo_accidente,tipo_vehiculo,tipo_persona,rango_edad,sexo,positiva_alcohol,positiva_droga
0,2022S000001,01/01/2022,1:30:00,13,Alcance,Turismo,Conductor,De 30 a 34 años,Mujer,N,NaN
1,2022S000001,01/01/2022,1:30:00,13,Alcance,Turismo,Conductor,De 45 a 49 años,Hombre,N,NaN
2,2022S000002,01/01/2022,0:30:00,3,Colisión fronto-lateral,Motocicleta hasta 125cc,Conductor,De 30 a 34 años,Hombre,S,NaN
3,2022S000002,01/01/2022,0:30:00,3,Colisión fronto-lateral,Motocicleta hasta 125cc,Pasajero,De 35 a 39 años,Mujer,N,NaN
4,2022S000002,01/01/2022,0:30:00,3,Colisión fronto-lateral,Turismo,Conductor,De 40 a 44 años,Hombre,N,NaN


Para poder tratar correctamente con los datos, vamos a convertir todos los datos posibles en números. Empezamos por positivo_alcohol y positivo_droga, que se cambiarán por 0 y 1.

Esta parte del código nos la propociona el compañero Luca
1. Sustituímos S por 1 y N por 0, Rellenamos los nulos en drogas por 0.
2. Sustituímos Hombre por 1 y Mujer por 0.

In [63]:
# Cambiamos en 'positiva_alcohol'
df_2022['positiva_alcohol'].replace({'S': 1, 'N': 0}, inplace=True)
df_2022['sexo'].replace({'Hombre': 1, 'Mujer': 0}, inplace=True)
# Cambiamos en 'positiva_droga' los nulls
df_2022['positiva_droga'].fillna(0, inplace=True)

df_2022.head()

,num_expediente,fecha,hora,cod_distrito,tipo_accidente,tipo_vehiculo,tipo_persona,rango_edad,sexo,positiva_alcohol,positiva_droga
0,2022S000001,01/01/2022,1:30:00,13,Alcance,Turismo,Conductor,De 30 a 34 años,0,0.0,0.0
1,2022S000001,01/01/2022,1:30:00,13,Alcance,Turismo,Conductor,De 45 a 49 años,1,0.0,0.0
2,2022S000002,01/01/2022,0:30:00,3,Colisión fronto-lateral,Motocicleta hasta 125cc,Conductor,De 30 a 34 años,1,1.0,0.0
3,2022S000002,01/01/2022,0:30:00,3,Colisión fronto-lateral,Motocicleta hasta 125cc,Pasajero,De 35 a 39 años,0,0.0,0.0
4,2022S000002,01/01/2022,0:30:00,3,Colisión fronto-lateral,Turismo,Conductor,De 40 a 44 años,1,0.0,0.0


### Géneros desconocidos
Aplicamos la inferencia de Luca para determinar si el conductor era hombre o mujer

### Tratamiento de fechas
Para que sea posible para nuestro modelo inferir de los datos, preferimos que las fechas y las horas se expresen en valores continuos.
Para lograrlo, vamos a traducir la fecha a minutos desde el 00:00. La fecha se traducirá en días desde el 1-1-20XX.

Necesitamos un dato extra que puede ser interesante agregar, festivos y fines de semana. Se añadirá esa columna con esa información relevante. Seguiremos la siguiente convención con los 14 días festivos anuales de la ciudad de Madrid, por lo que serán 10 festivos nacionales, 2 festivos autonómicos (Com. Madrid) y 2 festivos locales (Madrid).
* Se denotará día laboral por 0, y día festivo o de descanso por 1.


Calcular fines de semana y festivos.

Una rápida búsqueda por internet nos da los 14 días festivos. La fuente es el calendario de laboral de la Agencia Tributaria

In [64]:
from datetime import datetime

def is_work_day(date_str: str) -> int:
    date_format = '%d/%m/%Y'  # Formato para 'dd-MM-yyyy'
    
    holidays_madrid = [''] # todo
    try:
        date_obj = datetime.strptime(date_str, date_format)
        day_of_week_int = date_obj.weekday()
        if day_of_week_int == 5 or day_of_week_int == 6 or date_str in holidays_madrid:
            return 1
        return 0

    except ValueError:
        return -1

df_2022['bool_laborable'] = df_2022['fecha'].transform(is_work_day)

Calcular la hora a minutos desde las 00:00

In [65]:
def minutes_from_midnight(hour_str : str) -> int:
    clock = hour_str.split(':')
    minutes = 0
    
    minutes = int(clock[0]) * 60
    minutes = minutes + int(clock[1])
    
    return minutes

df_2022['hora'] = df_2022['hora'].transform(minutes_from_midnight)

In [68]:
df_2022.rename(columns={'hora': ' minutes_from_midnight'}, inplace=True)
df_2022

,num_expediente,fecha,minutes_from_midnight,cod_distrito,tipo_accidente,tipo_vehiculo,tipo_persona,rango_edad,sexo,positiva_alcohol,positiva_droga,bool_laborable
0,2022S000001,01/01/2022,90,13,Alcance,Turismo,Conductor,De 30 a 34 años,0,0.0,0.0,1
1,2022S000001,01/01/2022,90,13,Alcance,Turismo,Conductor,De 45 a 49 años,1,0.0,0.0,1
2,2022S000002,01/01/2022,30,3,Colisión fronto-lateral,Motocicleta hasta 125cc,Conductor,De 30 a 34 años,1,1.0,0.0,1
3,2022S000002,01/01/2022,30,3,Colisión fronto-lateral,Motocicleta hasta 125cc,Pasajero,De 35 a 39 años,0,0.0,0.0,1
4,2022S000002,01/01/2022,30,3,Colisión fronto-lateral,Turismo,Conductor,De 40 a 44 años,1,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
47048,2022S041289,29/12/2022,1180,11,Alcance,Turismo,Conductor,De 50 a 54 años,1,0.0,0.0,0
47049,2022S041289,29/12/2022,1180,11,Alcance,Turismo,Pasajero,De 50 a 54 años,0,0.0,0.0,0
47050,2022S041312,12/12/2022,1370,12,Colisión fronto-lateral,Turismo,Conductor,De 21 a 24 años,0,0.0,0.0,0
47051,2022S041312,12/12/2022,1370,12,Colisión fronto-lateral,Turismo,Conductor,De 35 a 39 años,1,0.0,0.0,0
